In [3]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import random
import itertools
import glob
import numpy as np

In [4]:
os.chdir('/Users/callamartyn/chou_lab/alongshan/out')

In [10]:
def count_genomes(path):
    df=pd.read_csv(path)
    df=df[df.cluster!='Cluster 8']
    if df.shape[1]==2:
        genomes=[]
    else:
        df=df.groupby('cluster').filter(lambda x: len(x) > 1)
        genomes=list(df[df.rep==True].contig_id)
    return genomes

In [11]:
files=glob.glob("./rarefaction/*_0??_cdhit.csv")

In [12]:
vlist=[count_genomes(x) for x in files]

In [13]:
no_samples=len(vlist)
df=pd.DataFrame(range(no_samples), columns=['no_samples'])
# rlist=[]
for i in range(50):
    random.shuffle(vlist)
    seen=[]
    no=0
    no_detected=[]
    for i2 in range(no_samples):
        for n in vlist[i2]:
            if n in seen:
                continue
            else:
                seen.append(n)
                no+=1
        no_detected.append(no)
#     rlist.append(no_detected)
    df['iteration_'+str(i)]=no_detected

In [16]:
df['median']=df.iloc[:,-50:].median(axis=1)
df['std']=df.iloc[:,-50:].std(axis=1)
df.to_csv('./rarefaction/sampling_samples.csv', index=False)

In [15]:
df

,no_samples,iteration_0,iteration_1,iteration_2,iteration_3,iteration_4,iteration_5,iteration_6,iteration_7,iteration_8,...,iteration_42,iteration_43,iteration_44,iteration_45,iteration_46,iteration_47,iteration_48,iteration_49,median,std
0,0,2,3,2,2,2,0,0,3,1,...,1,1,2,2,2,0,2,3,2.0,1.035098
1,1,4,4,2,4,3,0,1,4,2,...,1,3,4,3,4,2,3,3,3.0,1.373362
2,2,7,4,2,4,4,1,1,6,4,...,3,3,4,3,5,3,3,4,3.0,1.498843
3,3,7,5,4,6,4,1,1,7,5,...,5,5,5,3,5,3,4,4,4.0,1.633951
4,4,9,5,4,6,4,1,4,7,6,...,5,6,5,3,6,3,4,4,5.0,1.746834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,69,13,13,13,13,13,12,13,13,13,...,12,13,13,13,13,13,13,13,13.0,0.239898
70,70,13,13,13,13,13,12,13,13,13,...,12,13,13,13,13,13,13,13,13.0,0.239898
71,71,13,13,13,13,13,12,13,13,13,...,13,13,13,13,13,13,13,13,13.0,0.141421
72,72,13,13,13,13,13,12,13,13,13,...,13,13,13,13,13,13,13,13,13.0,0.141421


In [17]:
def chao_df(path):
    df=pd.read_csv(path)
    df=df.groupby('cluster').size().to_frame('size').reset_index()
    df['sample']=os.path.basename(path)[0:11]
    df['present']=np.where(df['size']>1 , 1, 0)
    df.drop(columns=['size'],inplace=True)
    return df


In [18]:
files=glob.glob("./rarefaction/*_0??_cdhit.csv")
p=[chao_df(x) for x in files]
df3=pd.concat(p)
# df3.pivot(index='cluster', columns='sample', values='present')
df3.to_csv('./rarefaction/presence_for_chao.csv',index=False)
# df.index

In [284]:
def pull_sum(path):
#     path='./rarefaction/tick001_092/TICK001_092_1M_cdhit.csv'
    sname=os.path.basename(path)[0:11]
    nreads=re.search(sname+'_(.*)_cdhit.csv',path).group(1)
    df=pd.read_csv(path)
    if df.shape[1]==2:
#         ngenomes=0
        genomes=[]
    else:
#         ngenomes=df[df.rep==False].cluster.nunique() # some genomes will be >1000bp but fragmented
        genomes=list(df[df.rep==False].cluster.unique())
    ngenomes=len(genomes)
    return(sname,nreads,ngenomes, genomes)


In [297]:
files=glob.glob("./rarefaction/sub_reads/*.csv")
# s,r,n, g=pull_sum(files[0])
df2=pd.DataFrame(columns=['sample', 'no_reads', 'no_genomes'])
for file in files:
    s,r,n,g=pull_sum(file)
    df2=df2.append({'sample':s,'no_reads':r,'no_genomes':n, 'genomes':g},ignore_index=True)
df2['no_reads']=[int(x)*2 for x in df2.no_reads]
df2.to_csv('./rarefaction/sampling_reads.csv',index=False)
df2

,sample,no_reads,no_genomes,genomes
0,TICK001_022,4378680,2,"[Cluster 2, Cluster 4]"
1,TICK001_018,200000,3,"[Cluster 3, Cluster 5, Cluster 8]"
2,TICK001_032,200000,2,"[Cluster 7, Cluster 8]"
3,TICK001_092,2000000,3,"[Cluster 8, Cluster 14, Cluster 17]"
4,TICK001_054,4000000,2,"[Cluster 8, Cluster 10]"
5,TICK001_029,200000,1,[Cluster 3]
6,TICK001_032,4000000,4,"[Cluster 0, Cluster 7, Cluster 8, Cluster 9]"
7,TICK001_022,200000,1,[Cluster 4]
8,TICK001_037,4000000,2,"[Cluster 2, Cluster 8]"
9,TICK001_037,1000000,1,[Cluster 8]


In [321]:

def readt18(path):
    df=pd.read_csv(path)
    sname=os.path.basename(path)[0:11]
    nreads=re.search(sname+'_(.*)_cdhit.csv',path).group(1)
    df=df.assign(nreads=nreads)
    return df



,cluster,length,contig_id,per_identity,rep,nreads
0,Cluster 0,11875,North_Fork_virus,100.00,True,100000
1,Cluster 1,11738,Doud_Peak_virus,100.00,True,100000
2,Cluster 2,10808,Rocky_Ridge_virus,100.00,True,100000
3,Cluster 3,10509,Lobos_virus,100.00,True,100000
4,Cluster 3,10475,NODE_1_length_10475_cov_138.944050,99.85,False,100000
...,...,...,...,...,...,...
20,Cluster 14,2315,NODE_27_length_2315_cov_5.566814,99.83,False,1000000
21,Cluster 15,2621,Notleys_Landing_Virus_str_2,100.00,True,1000000
22,Cluster 16,2593,Portuguese_Ridge_virus,100.00,True,1000000
23,Cluster 17,1796,Castle_Rock_Virus,100.00,True,1000000


In [346]:
files=glob.glob("./rarefaction/sub_reads/t18/TICK001_018*.csv")
t18=[readt18(x) for x in files]
t18=pd.concat(t18)

namedict=dict(zip(t18[t18.rep==True].cluster,t18[t18.rep==True].contig_id))
t18_g=list(t18[t18.rep==False].cluster.unique())
t18=t18[t18.rep==False]
t18['nreads']=[int(x)*2 for x in t18.nreads]


In [347]:
t18=t18.groupby(['cluster','nreads']).agg({'length':max}).reset_index()
t18['virus']=t18.cluster.map(namedict)
t18.to_csv('./rarefaction/tick18_frag.csv',index=False)
t18

,cluster,nreads,length,virus
0,Cluster 0,200000,241,North_Fork_virus
1,Cluster 0,1000000,696,North_Fork_virus
2,Cluster 0,2000000,519,North_Fork_virus
3,Cluster 0,4000000,761,North_Fork_virus
4,Cluster 0,4616016,761,North_Fork_virus
5,Cluster 12,200000,264,Joshua_Creek_virus
6,Cluster 13,200000,441,Kasler_Point_virus
7,Cluster 13,1000000,3153,Kasler_Point_virus
8,Cluster 13,2000000,3175,Kasler_Point_virus
9,Cluster 14,200000,249,Garrapata_Virus_str


Mosquitos